#KNN

###Objetivo
Predecir con KNN la hipotesis\
Hipótesis de clasificación:
"*¿Es posible predecir si un jugador sobrevivirá en una ronda?*"

##Importaciones

In [26]:
# Manipulación de datos
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV


##Carga de los datos

In [2]:
from google.colab import files
import pandas as pd # Importing the pandas library

uploaded = files.upload()
for fn in uploaded.keys():
    name=fn
df = pd.read_csv(name, sep=";") # Now pd is defined and can be used

Saving Anexo ET_demo_round_traces_2022.csv to Anexo ET_demo_round_traces_2022.csv


/tmp/ipython-input-2-3048744678.py:7: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(name, sep=";") # Now pd is defined and can be used


In [3]:
df.head()

,Unnamed: 0,Map,Team,InternalTeamId,MatchId,RoundId,RoundWinner,MatchWinner,Survived,AbnormalMatch,...,RoundKills,RoundAssists,RoundHeadshots,RoundFlankKills,RoundStartingEquipmentValue,TeamStartingEquipmentValue,MatchKills,MatchFlankKills,MatchAssists,MatchHeadshots
0,600,de_inferno,Terrorist,1,4,1,False,True,False,False,...,0,0,0,0,750,4400,0,0,0,0
1,601,de_inferno,Terrorist,1,4,1,False,True,False,False,...,0,0,0,0,800,4400,0,0,0,0
2,602,de_inferno,Terrorist,1,4,1,False,True,False,False,...,0,0,0,0,1000,4400,0,0,0,0
3,603,de_inferno,Terrorist,1,4,1,False,True,False,False,...,0,0,0,0,850,4400,0,0,0,0
4,604,de_inferno,Terrorist,1,4,1,False,True,True,False,...,0,0,0,0,1000,4400,0,0,0,0


In [4]:
df_copy = df.copy()

##Limpieza de datos



In [5]:
#vemos si hay datos faltantes
df_copy.columns[df.isnull().any()]

Index(['Team', 'MatchWinner'], dtype='object')

In [6]:
# Verifiquemos el porcentaje de valores faltantes en estas columnas.
miss = df_copy.isnull().sum()/len(df_copy)
miss = miss[miss > 0]
miss.sort_values(inplace=True)
miss

,0
MatchWinner,0.000013
Team,0.000025


In [7]:
#como son pocos, eliminamos las filas con valores nulos
# Eliminar todas las filas que contengan al menos un valor nulo
df_copy = df_copy.dropna()

# Confirmar cuántas filas quedaron
print(f"Filas restantes después de eliminar nulos: {len(df_copy)}")


Filas restantes después de eliminar nulos: 79154


In [9]:
#conversión de columnas a tipo booleano primero, luego a int (0 o 1)
cols = ['RoundWinner', 'MatchWinner', 'Survived', 'AbnormalMatch']

for col in cols:
    df_copy[col] = df_copy[col].astype('bool').astype('int')


In [10]:
df_copy.head()

,Unnamed: 0,Map,Team,InternalTeamId,MatchId,RoundId,RoundWinner,MatchWinner,Survived,AbnormalMatch,...,RoundKills,RoundAssists,RoundHeadshots,RoundFlankKills,RoundStartingEquipmentValue,TeamStartingEquipmentValue,MatchKills,MatchFlankKills,MatchAssists,MatchHeadshots
0,600,de_inferno,Terrorist,1,4,1,1,1,0,0,...,0,0,0,0,750,4400,0,0,0,0
1,601,de_inferno,Terrorist,1,4,1,1,1,0,0,...,0,0,0,0,800,4400,0,0,0,0
2,602,de_inferno,Terrorist,1,4,1,1,1,0,0,...,0,0,0,0,1000,4400,0,0,0,0
3,603,de_inferno,Terrorist,1,4,1,1,1,0,0,...,0,0,0,0,850,4400,0,0,0,0
4,604,de_inferno,Terrorist,1,4,1,1,1,1,0,...,0,0,0,0,1000,4400,0,0,0,0


In [11]:
# Aplicar One-Hot Encoding a la columna 'Team'
df_copy = pd.get_dummies(df_copy, columns=['Team'], drop_first=True)

# Verificar qué columnas se crearon
team_cols = [col for col in df.columns if col.startswith('Team_')]
print("Columnas creadas para Team:", team_cols)


Columnas creadas para Team: []


In [12]:
df_copy['Map'].unique()


array(['de_inferno', 'de_nuke', 'de_mirage', 'de_dust2'], dtype=object)

In [13]:
# Aplicar One-Hot Encoding a la columna 'Map'
df_copy = pd.get_dummies(df_copy, columns=['Map'], drop_first=True)

# Verifica los nuevos nombres de columnas
print(df_copy.columns)


Index(['Unnamed: 0', 'InternalTeamId', 'MatchId', 'RoundId', 'RoundWinner',
       'MatchWinner', 'Survived', 'AbnormalMatch', 'TimeAlive',
       'TravelledDistance', 'RLethalGrenadesThrown',
       'RNonLethalGrenadesThrown', 'PrimaryAssaultRifle', 'PrimarySniperRifle',
       'PrimaryHeavy', 'PrimarySMG', 'PrimaryPistol', 'FirstKillTime',
       'RoundKills', 'RoundAssists', 'RoundHeadshots', 'RoundFlankKills',
       'RoundStartingEquipmentValue', 'TeamStartingEquipmentValue',
       'MatchKills', 'MatchFlankKills', 'MatchAssists', 'MatchHeadshots',
       'Team_Terrorist', 'Map_de_inferno', 'Map_de_mirage', 'Map_de_nuke'],
      dtype='object')


cuando las 3 columnas nuevas de mapa sean =0, entonces el mapa es de dust2

In [14]:
# Selecciona todas las columnas que son booleanas
bool_cols = ['Team_Terrorist', 'Map_de_inferno', 'Map_de_mirage', 'Map_de_nuke']

# Convierte True/False a 1/0
df_copy[bool_cols] = df_copy[bool_cols].astype(int)

# Verificación
print(df_copy[bool_cols].head())


   Team_Terrorist  Map_de_inferno  Map_de_mirage  Map_de_nuke
0               1               1              0            0
1               1               1              0            0
2               1               1              0            0
3               1               1              0            0
4               1               1              0            0


In [15]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79154 entries, 0 to 79156
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   79154 non-null  int64  
 1   InternalTeamId               79154 non-null  int64  
 2   MatchId                      79154 non-null  int64  
 3   RoundId                      79154 non-null  int64  
 4   RoundWinner                  79154 non-null  int64  
 5   MatchWinner                  79154 non-null  int64  
 6   Survived                     79154 non-null  int64  
 7   AbnormalMatch                79154 non-null  int64  
 8   TimeAlive                    79154 non-null  object 
 9   TravelledDistance            79154 non-null  object 
 10  RLethalGrenadesThrown        79154 non-null  int64  
 11  RNonLethalGrenadesThrown     79154 non-null  int64  
 12  PrimaryAssaultRifle          79154 non-null  float64
 13  PrimarySniperRifle   

In [16]:
# eliminao

df_copy = df_copy.drop('FirstKillTime', axis=1)

In [17]:
# Limpiar TimeAlive
df_copy['TimeAlive'] = (
    df['TimeAlive']
    .astype(str)
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
    .astype(float)
)

# Limpiar TravelledDistance (maneja notación científica con coma)
df_copy['TravelledDistance'] = (
    df['TravelledDistance']
    .astype(str)
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
    .astype(float)
)


In [18]:
correlation = df_copy.corr()['Survived'].sort_values(ascending=False)

# Filtrar las 10 variables más correlacionadas con MatchKills
top_10_correlations = correlation.head(11)
print(top_10_correlations)

Survived                       1.000000
RoundKills                     0.331627
RoundHeadshots                 0.205295
TeamStartingEquipmentValue     0.173214
RLethalGrenadesThrown          0.171730
RoundStartingEquipmentValue    0.170569
RoundFlankKills                0.139682
RNonLethalGrenadesThrown       0.125091
PrimaryAssaultRifle            0.110011
PrimarySniperRifle             0.067108
MatchKills                     0.059466
Name: Survived, dtype: float64


In [19]:
# Lista de columnas a conservar (features + target)
columnas = [

'RoundKills',
'RoundHeadshots',
'TeamStartingEquipmentValue',
'RLethalGrenadesThrown',
'RoundStartingEquipmentValue',
'RoundFlankKills',
'RNonLethalGrenadesThrown',
'PrimaryAssaultRifle',
'PrimarySniperRifle',
'MatchKills',
'Survived' #target
]

# Crear nuevo DataFrame
df2 = df_copy[columnas].copy()

# Mostrar primeras filas para confirmar
df2.head()

,RoundKills,RoundHeadshots,TeamStartingEquipmentValue,RLethalGrenadesThrown,RoundStartingEquipmentValue,RoundFlankKills,RNonLethalGrenadesThrown,PrimaryAssaultRifle,PrimarySniperRifle,MatchKills,Survived
0,0,0,4400,0,750,0,4,0.0,0.0,0,0
1,0,0,4400,0,800,0,0,0.0,0.0,0,0
2,0,0,4400,0,1000,0,0,0.0,0.0,0,0
3,0,0,4400,0,850,0,0,0.0,0.0,0,0
4,0,0,4400,1,1000,0,1,0.0,0.0,0,1


In [20]:
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
RoundKills,79154.0,0.671362,0.881808,0.0,0.0,0.0,1.0,5.0
RoundHeadshots,79154.0,0.301564,0.578851,0.0,0.0,0.0,0.0,5.0
TeamStartingEquipmentValue,79154.0,18890.255704,9743.166727,0.0,10900.0,22550.0,26050.0,36150.0
RLethalGrenadesThrown,79154.0,0.498673,0.597989,0.0,0.0,0.0,1.0,4.0
RoundStartingEquipmentValue,79154.0,3778.153347,2111.287720,0.0,1600.0,4700.0,5200.0,8850.0
RoundFlankKills,79154.0,0.095636,0.320646,0.0,0.0,0.0,0.0,5.0
RNonLethalGrenadesThrown,79154.0,1.258104,1.184906,0.0,0.0,1.0,2.0,6.0
PrimaryAssaultRifle,79154.0,0.675376,0.464120,0.0,0.0,1.0,1.0,1.0
PrimarySniperRifle,79154.0,0.089844,0.279801,0.0,0.0,0.0,0.0,1.0
MatchKills,79154.0,8.511850,6.195036,0.0,3.0,8.0,13.0,41.0


##Entrenar modelo


In [21]:
X = df2.drop('Survived', axis=1)
y = df2['Survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Escalar es muy importante en KNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
# Probar con k=5 (valor común)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

##Prediccion y evaluacion

In [23]:
# Predicción
y_pred = knn.predict(X_test_scaled)

In [24]:
#Evaluacion
# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Reporte detallado
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("Matriz de confusión")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.6413366180279199
              precision    recall  f1-score   support

           0       0.68      0.74      0.71      9443
           1       0.56      0.50      0.53      6388

    accuracy                           0.64     15831
   macro avg       0.62      0.62      0.62     15831
weighted avg       0.64      0.64      0.64     15831

Matriz de confusión
[[6978 2465]
 [3213 3175]]


GridSearch

In [27]:
param_grid = {
    'n_neighbors': list(range(1, 21)),  # probar k de 1 a 20
    'weights': ['uniform', 'distance']
}

grid = GridSearchCV(
    KNeighborsClassifier(),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train_scaled, y_train)

print("Mejores parámetros:", grid.best_params_)

# Evaluación del mejor modelo
best_knn = grid.best_estimator_
y_pred = best_knn.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Mejores parámetros: {'n_neighbors': 20, 'weights': 'uniform'}
Accuracy: 0.663381972080096
              precision    recall  f1-score   support

           0       0.68      0.81      0.74      9443
           1       0.62      0.44      0.51      6388

    accuracy                           0.66     15831
   macro avg       0.65      0.63      0.63     15831
weighted avg       0.66      0.66      0.65     15831

Matriz de confusión:
[[7695 1748]
 [3581 2807]]
